In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step 1: 환경 설정 및 라이브러리 임포트**

In [9]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 딥러닝(GRU) 라이브러리
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 경로 설정
DATA_DIR = Path("/content/drive/MyDrive/COSE362/data/feature_engineering")
OUTPUT_DIR = Path("/content/drive/MyDrive/COSE362/data/prediction_output")
OUTPUT_DIR.mkdir(exist_ok=True)

# GPU 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

Using Device: cuda


**Step 2: 데이터 로드 및 날짜 분할 함수 정의**

In [10]:
def load_and_split_data(file_path):
    """
    Parquet 파일을 로드하고 날짜 기준으로 Train/Valid/Test로 분할
    Train: 2017-01-01 ~ 2018-12-31
    Valid: 2019-01-01 ~ 2019-06-30
    Test:  2019-07-01 ~ 2019-12-31
    """
    print(f"Loading {file_path}...")
    df = pd.read_parquet(file_path)

    # 1. 날짜순 정렬
    if 'date_index' in df.columns:
        df = df.sort_values('date_index').reset_index(drop=True)

    # 2. 날짜 열(datetime 객체) 생성
    # feature_engineering에서 만든 'date_str' (예: 2017_01_01)을 이용
    if 'date_str' in df.columns:
        df['Date'] = pd.to_datetime(df['date_str'], format='%Y_%m_%d')
    elif 'article_date' in df.columns:
        df['Date'] = pd.to_datetime(df['article_date'])
    else:
        # 만약 날짜 열이 없다면 에러 발생보다는 인덱스로 대체
        print("Warning: Date column not found. Using index based splitting.")
        # 임시 날짜 생성
        df['Date'] = pd.date_range(start='2017-01-01', periods=len(df), freq='D')

    # 3. 날짜 기준 분할
    train_mask = (df['Date'] >= '2017-01-01') & (df['Date'] <= '2018-12-31')
    valid_mask = (df['Date'] >= '2019-01-01') & (df['Date'] <= '2019-06-30')
    test_mask  = (df['Date'] >= '2019-07-01') & (df['Date'] <= '2019-12-31')

    train_df = df[train_mask].copy()
    valid_df = df[valid_mask].copy()
    test_df  = df[test_mask].copy()

    # 4. 결과 출력
    print(f"Train: {train_df.shape}, Valid: {valid_df.shape}, Test: {test_df.shape}")

    if len(train_df) == 0:
        print("!!! Warning: Train set is empty. Check date format !!!")

    return train_df, valid_df, test_df

**Step 3: Feature & Target 준비**

In [11]:
def prepare_xy(df, target_col='value'):
    """
    데이터프레임에서 학습에 방해되는 메타데이터를 제거하고
    X(입력 Feature)와 y(정답 Target)를 분리
    """
    # ---------------------------------------------------------
    # 1. Target 생성 (내일의 주가를 예측 목표로 설정)
    # ---------------------------------------------------------
    # 오늘 행에 내일 주가(target)를 정답지로 붙임
    df['target'] = df[target_col].shift(-1)

    # 마지막 날은 내일 데이터가 없으므로(NaN) 삭제함
    df = df.dropna(subset=['target'])

    # ---------------------------------------------------------
    # 2. 제거할 열 정의
    # ---------------------------------------------------------
    # 모델은 오직 '숫자'만 계산할 수 있어서
    # 일부 열들은 학습에서 제외해야 함
    cols_to_drop = [
        # [예측 목표]
        'target',               # 정답지는 입력(X)에서 빼기

        # [날짜 관련]
        'Date',                 # load_and_split_data에서 생성된 datetime 객체
        'date_str',             # '2017_01_01' (문자열)
        'article_date',         # datetime 객체
        'pub_date',             # '2017_12_31' (문자열)
        'date_index',           # 0, 1, 2... (단순 인덱스)

        # [기사/인물 메타데이터 (문자열/ID)]
        'person',               # 'alex_morgan' (문자열)
        'person_id',            # 1, 2... (숫자지만 One-hot 벡터와 중복됨)
        'article_id',           # 기사 고유 ID (단순 인덱스)
        'headline',             # 기사 제목 (문자열)
        'idx',                  # 임베딩 파일 내부 인덱스

        # [기타]
        'embedding'             # 혹시 '_orig' 파일을 쓸 경우, 리스트 형태라 모델에 바로 못 넣음
    ]

    # 실제 데이터프레임에 존재하는 열만 골라서 제거 (에러 방지)
    actual_drop_cols = [c for c in cols_to_drop if c in df.columns]

    # ---------------------------------------------------------
    # 3. X, y 분리
    # ---------------------------------------------------------
    X = df.drop(columns=actual_drop_cols)
    y = df['target']

    # ---------------------------------------------------------
    # 모델에 들어가는 최종 Features
    # 1. value (오늘 주가)
    # 2. lag_1 ~ lag_5 (과거 주가)
    # 3. pca_0 ~ pca_N (뉴스 임베딩 특징)
    # 4. person_1 ~ person_100 (인물 One-hot)
    # 5. fg_value, fg_lag_* (공포지수)
    # ---------------------------------------------------------

    return X, y, df['Date'] # 날짜 정보는 나중에 그래프 그릴 때 활용 가능해서 같이 반환

### **[모델 1: Linear Regression]**

In [12]:
import gc
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

def run_regularized_regression(dataset_name, file_path, model_type="ridge", alpha=1.0):
    print(f"\n=== Running {model_type.upper()} (alpha={alpha}) on {dataset_name} ===")

    # 1. 데이터 로드
    train_df, valid_df, test_df = load_and_split_data(file_path)

    # 2. X, y 준비
    X_train, y_train, _ = prepare_xy(train_df)
    X_test, y_test, dates_test = prepare_xy(test_df)

    # 메모리 확보
    del train_df, valid_df, test_df
    gc.collect()

    # 데이터 타입 다운캐스팅
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # 3. 모델 정의
    if model_type == "ridge":
        model = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    elif model_type == "lasso":
        model = make_pipeline(StandardScaler(), Lasso(alpha=alpha))
    else:
        model = make_pipeline(StandardScaler(), LinearRegression())

    # 4. 학습
    model.fit(X_train, y_train)

    # 학습 데이터 삭제
    del X_train, y_train
    gc.collect()

    # 5. 예측
    preds_test = model.predict(X_test)

    # 6. 평가
    mse = mean_squared_error(y_test, preds_test)
    print(f"Test MSE: {mse:.4f}")

    # 7. 결과 저장
    result_data = []
    for date, true, pred in zip(dates_test, y_test, preds_test):
        result_data.append({
            "date": date.strftime('%Y-%m-%d'),
            "actual": float(true),
            "predicted": float(pred)
        })

    with open(OUTPUT_DIR / f"pred_{model_type}_{dataset_name}.json", "w") as f:
        json.dump(result_data, f, indent=4)

    del model, X_test, y_test, preds_test
    gc.collect()

    return mse

### **[모델 2: GRU(Deep Learning)]**

In [13]:
from torch.utils.data import Dataset

# 1. Dataset 정의 (Lazy Loading)
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_length=5):
        # 데이터를 미리 텐서(float32)로 변환하여 저장
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, i):
        # 호출될 때마다 슬라이싱해서 반환
        return self.X[i : i + self.seq_length], self.y[i + self.seq_length]

# 2. GRU 모델 정의
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=1, num_layers=2):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# 3. run_gru 함수
def run_gru(dataset_name, file_path, seq_length=5, epochs=30): # Epochs 30으로 조정
    print(f"\n=== Running GRU on {dataset_name} ===")

    # --- 데이터 로드 ---
    train_df, valid_df, test_df = load_and_split_data(file_path)

    # --- 전처리 (Numpy 변환) ---
    # DataFrame 상태로 두지 않고 바로 Numpy로 변환
    X_train_raw, y_train_raw, _ = prepare_xy(train_df)
    X_valid_raw, y_valid_raw, _ = prepare_xy(valid_df)
    X_test_raw, y_test_raw, dates_test = prepare_xy(test_df)

    del train_df, valid_df, test_df
    gc.collect()

    # --- 스케일링 ---
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_scaled = scaler_x.fit_transform(X_train_raw).astype(np.float32)
    y_train_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1)).astype(np.float32)

    X_valid_scaled = scaler_x.transform(X_valid_raw).astype(np.float32)
    y_valid_scaled = scaler_y.transform(y_valid_raw.values.reshape(-1, 1)).astype(np.float32)

    X_test_scaled = scaler_x.transform(X_test_raw).astype(np.float32)
    y_test_raw_values = y_test_raw.values.astype(np.float32) # 정답 비교용

    # --- Dataset & DataLoader (Lazy Loading 적용) ---
    train_dataset = TimeSeriesDataset(X_train_scaled, y_train_scaled, seq_length)
    # num_workers=0 설정
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)

    # Test용 Dataset (shuffle=False)
    test_dataset = TimeSeriesDataset(X_test_scaled, y_test_raw_values, seq_length) # y값은 dummy로 넣어도 됨
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0)

    # --- 모델 초기화 ---
    input_dim = X_train_scaled.shape[1]
    model = GRUModel(input_dim).to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # --- 학습 루프 ---
    model.train()
    for epoch in range(epochs):
        for bx, by in train_loader:
            bx, by = bx.to(DEVICE), by.to(DEVICE)

            optimizer.zero_grad()
            output = model(bx)
            loss = criterion(output, by)
            loss.backward()
            optimizer.step()

        # 로그 출력 (10 epoch 마다)
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

    # --- 예측 (Batch Processing) ---
    model.eval()
    preds_list = []

    with torch.no_grad():
        for bx, _ in test_loader:
            bx = bx.to(DEVICE)
            out = model(bx)
            preds_list.append(out.cpu().numpy())

    # 결과 합치기
    test_preds_scaled = np.vstack(preds_list)
    test_preds = scaler_y.inverse_transform(test_preds_scaled)

    # 정답 데이터 길이 맞춤 (앞부분 seq_length 만큼 잘림)
    y_test_true = y_test_raw_values[seq_length:]
    test_dates_clipped = dates_test.values[seq_length:]

    # --- 평가 ---
    mse = mean_squared_error(y_test_true, test_preds)
    print(f"Test MSE: {mse:.4f}")

    # --- 저장 ---
    result_data = []
    for date, true, pred in zip(test_dates_clipped, y_test_true, test_preds):
        date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        result_data.append({
            "date": date_str,
            "actual": float(true),
            "predicted": float(pred[0])
        })

    with open(OUTPUT_DIR / f"pred_gru_{dataset_name}.json", "w") as f:
        json.dump(result_data, f, indent=4)

    del model, optimizer, train_loader, test_loader, train_dataset, test_dataset
    del X_train_scaled, X_valid_scaled, X_test_scaled
    torch.cuda.empty_cache()
    gc.collect()

    return mse

**Step 4: 전체 실행 및 결과 집계**

In [14]:
import gc

# ====================================================
# Main Execution Loop: All Datasets
# ====================================================

levels = ['B', 'C', 'D']
methods = ['headlines', 'chunking', 'bodyText', 'paragraphs']
types = ['pca']

metrics_data = []

# --- Baseline: Dataset A ---
file_path_A = DATA_DIR / "dataset_A.parquet"
if file_path_A.exists():
    print(f"\n>>> Processing Baseline: Dataset A")
    mse_ridge = run_regularized_regression("Dataset_A", file_path_A, model_type="ridge")
    metrics_data.append({"Dataset": "A", "Method": "-", "Type": "-", "Model": "Ridge", "MSE": mse_ridge})

    mse_gru = run_gru("Dataset_A", file_path_A)
    metrics_data.append({"Dataset": "A", "Method": "-", "Type": "-", "Model": "GRU", "MSE": mse_gru})

    gc.collect()
    torch.cuda.empty_cache()
else:
    print("Warning: dataset_A.parquet not found.")


# --- Main Loop ---
for level in levels:
    for method in methods:
        for dtype in types:
            filename = f"dataset_{level}_{method}_{dtype}.parquet"
            file_path = DATA_DIR / filename
            dataset_name = f"{level}_{method}_{dtype}"

            if not file_path.exists():
                print(f"Skipping {filename}: File not found.")
                continue

            print(f"\n>>> Processing {filename} ...")

            try:
                # 1. Ridge
                mse_ridge = run_regularized_regression(dataset_name, file_path, model_type="ridge")
                metrics_data.append({
                    "Dataset": level, "Method": method, "Type": dtype,
                    "Model": "Ridge", "MSE": mse_ridge
                })

                # 2. GRU
                mse_gru = run_gru(dataset_name, file_path, seq_length=5, epochs=30)
                metrics_data.append({
                    "Dataset": level, "Method": method, "Type": dtype,
                    "Model": "GRU", "MSE": mse_gru
                })

            except Exception as e:
                print(f"Error processing {filename}: {e}")

            finally:
                gc.collect()
                torch.cuda.empty_cache()

# --- 최종 결과 저장 ---
metrics_df = pd.DataFrame(metrics_data)
metrics_df = metrics_df.sort_values(by=["Dataset", "Method", "Model"])
metrics_df.to_csv(OUTPUT_DIR / "final_evaluation_metrics.csv", index=False)

print("\n=== Final Experiment Results (Sample) ===")
print(metrics_df.head(10))
print(f"\nSaved to: {OUTPUT_DIR / 'final_evaluation_metrics.csv'}")


>>> Processing Baseline: Dataset A

=== Running RIDGE (alpha=1.0) on Dataset_A ===
Loading /content/drive/MyDrive/COSE362/data/feature_engineering/dataset_A.parquet...
Train: (502, 9), Valid: (124, 9), Test: (128, 9)
Test MSE: 480.3128

=== Running GRU on Dataset_A ===
Loading /content/drive/MyDrive/COSE362/data/feature_engineering/dataset_A.parquet...
Train: (502, 9), Valid: (124, 9), Test: (128, 9)
Epoch 10/30, Loss: 0.006328
Epoch 20/30, Loss: 0.003206
Epoch 30/30, Loss: 0.002886
Test MSE: 3823.1785

>>> Processing dataset_B_headlines_pca.parquet ...

=== Running RIDGE (alpha=1.0) on B_headlines_pca ===
Loading /content/drive/MyDrive/COSE362/data/feature_engineering/dataset_B_headlines_pca.parquet...
Train: (312535, 417), Valid: (76232, 417), Test: (72503, 417)
Test MSE: 0.7667

=== Running GRU on B_headlines_pca ===
Loading /content/drive/MyDrive/COSE362/data/feature_engineering/dataset_B_headlines_pca.parquet...
Train: (312535, 417), Valid: (76232, 417), Test: (72503, 417)
Epoch 